In [1]:
import numpy as np
import pandas as pd

## Extract scores

We used Gradescope for all homework and exams. Gradescope lets you export all grades in one large CSV which contains plenty of metadata such as submission times.

In this notebook we extract the student grades from the CSV in preparation for the statistical analysis performed on a separate notebook. We also shuffle the group and section numbers to preserve the students' anonymity.

The original CSV has been removed before uploading to a public GitHub repository, also for anonymity.

In [2]:
grades_list = []

n_hw = 6 #number of homeworks
n_qz = 6 #number of quizzes
sections = [10, 14, 15]

hw_list = [f"Homework {k+1}" for k in range(n_hw)]
qz_list = [f"Quiz {k+1}" for k in range(n_qz)]
cols = hw_list + qz_list + ["Final Exam"]

rename = {
    "Homework #5": "Homework 5",
    "Quiz #2": "Quiz 2",
    "Assignment 1": "Homework 1",
    "Assignment 2": "Homework 2",
    "Assignment 3": "Homework 3",
    "Assignment 4": "Homework 4",
    "Assignment 5": "Homework 5",
    "Assignment 6": "Homework 6",
}

students_dropped = [] # Taken out for anonymity


for n in sections:
    grades = (pd.read_csv(f"./data/section_{n}_grades.csv")
               .rename(columns=rename)
               .fillna(0)
               .set_index('Email'))
    groups = (pd.read_csv(f'./data/section{n}groups.csv')
                .set_index('Email'))
    grades = grades[cols]
    groups = groups['Group']
    grades['Section number'] = f'Section {n}'
    keep = [email for email in grades.index if email not in students_dropped]
    grades = (pd.merge(grades, groups, on='Email')
               .loc[keep])
    grades_list.append(grades)
    
all_grades = pd.concat(grades_list) # More efficient to concatenate all at once (in this case speed is not an issue but this is good practice)

Now we will compute the final score. The lowest score in both quizzes and homeworks are dropped.

In [3]:
all_grades['Total'] = all_grades[hw_list].apply(lambda r: r.sum()-r.min(),  axis=1) + all_grades[qz_list].apply(lambda r: r.sum()-r.min(), axis=1) + all_grades['Final Exam']
all_grades['Percentage'] = all_grades['Total'] / 5

## Some students got an extra "drop" (removed for anonymity)
# special_cases = []
# all_grades.loc[special_cases, 'Total'] = all_grades[hw_list].apply(lambda r: r.sum() - r.nsmallest(2).sum(), axis=1) + all_grades[qz_list].apply(lambda r: r.sum() - r.nsmallest(2).sum(), axis=1) + all_grades['Final Exam']
# all_grades.loc[special_cases, 'Percentage'] = all_grades.loc[special_cases, 'Total'] / 4.6

In the following cell I permute the group names to anonymize the data.

In [4]:
# Removed the group comprised of the students who were in China and
# had their own group (for time zone reasons), since that group was not random
all_grades_anonymized = all_grades.loc[~(all_grades['Group']=='China'), ['Section number', 'Group', 'Percentage']].copy()

# Shuffle group numbers
for n in sections:
    this_section = all_grades_anonymized['Section number'] == f'Section {n}'
    groups = all_grades_anonymized.loc[this_section, 'Group'].unique()
    all_grades_anonymized.loc[this_section, 'Group'] = all_grades_anonymized.loc[this_section, 'Group'].replace(groups, np.random.permutation(groups))

# Shuffle students (doesn't affect group membership)
all_grades_anonymized = all_grades_anonymized.sample(frac=1).reset_index(drop=True)

# Rename sections
sections = all_grades_anonymized['Section number'].unique()
shuffled_sections = np.random.permutation(sections)
all_grades_anonymized['Section number'] = all_grades_anonymized['Section number'].replace(shuffled_sections, [1, 2, 3])

In [5]:
# Drop the one group with only 2 students
# to_drop = ....
# all_grades_anonymized = all_grades_anonymized[~to_drop].copy()